In [166]:
%matplotlib inline

In [183]:
# Storage paths for the Crops and the Panos
pano_file_directory = "/Users/marcus/Desktop/Training_Data/Pano_4096_2048/" 
crop_file_directory = "/Users/marcus/Desktop/Training_Data/Crops_From_4096_2048"
csv_out_file_directory = "/Users/marcus/Desktop/Training_Data/CSV_Output"

In [181]:
import os
import numpy as np
from skimage import io
from skimage.feature import match_template
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import json
import math
import csv

In [176]:
# Parses through the pano json file, and creates Pano Objects 

pano_object_list = []

for dir_name, sub_dir_list, file_list in os.walk(pano_file_directory):
    for file_name in file_list:
        if file_name[-5:] == ".json" and len(file_name) > 20:
            
            new_pano_object = pano_object(file_name, "_4096_2048")
            pano_object_list.append(new_pano_object)

In [177]:
class pano_object():
    
    pano_image_width = 4096
    pano_image_height = 2048
    
    
    def __init__(self, json_file, size_extension = ""):
        self.name = json_file[:-21]
        self.json_path = os.path.join(pano_file_directory, self.name + size_extension + "_label.json")
        self.pano_path = os.path.join(pano_file_directory, self.name + size_extension + ".jpg")
        self.labels = []
        
        with open(self.json_path, 'r') as pano_json:
            label_data = json.load(pano_json)
            for label in label_data[self.name]:
                new_label = label_object(label, self.name)
                self.labels.append(new_label)
        self.label_size = len(self.labels)
    
    def get_pano_image(self):
        pano = io.imread(self.pano_path)[:,:,2]
        return pano
    
    
class label_object():
    
    def __init__(self, label_data, pano_name):
        self.id = label_data['label_id']
        self.type = label_data['label_type']
        self.x = label_data['x']
        self.y = label_data['y']
        self.pano_name = pano_name
        crop_file_name = "{}_._{}_._{}_._{}_._.jpg".format(pano_name, label_data['label_type'], label_data['x'], label_data['y'] )
        self.label_crop_path = os.path.join(crop_file_directory, label_data['label_type'], pano_name[:2], crop_file_name)
        
    def get_label_crop_image(self):
        crop = io.imread(self.label_crop_path)[:,:,2]
        return crop
    
    def get_distance(self, x, y):
        x1 = min(self.x, x)
        x2 = max(self.x, x)
        
        dx = min((x2 - x1), (pano_object.pano_image_width - x2 + x1))
        dy = self.y - y
        
        distance = math.sqrt((dx)**2 + (dy)**2)
        
        return distance
        

In [179]:
def compare_labels(test_label, train_label):
    if test_label.type == train_label.type:
        return True
    else:
        return False

In [ ]:
data = np.array(pano_object_list, dtype = np.object)

kfold = KFold(2, False, 4)

k = 0
for train, test in kfold.split(data):
    csv_file_name = "distance_type_calculation_" + str(k) + ".csv"
    csv_file_path = os.path.join(csv_out_file_directory, csv_file_name)
    k += 1
    with open(csv_file_path, 'w') as csvfile:
        field_names = ['target_pano_id', 'test_pano_id', 'target_label_id', 'test_label_id', 'target_x', 'target_y', 'test_x_pred', 'test_y_pred', 'target_type', 'test_type', 'distance', 'is_same_type' ]
        writer = csv.DictWriter(csvfile, fieldnames = field_names)
        writer.writeheader()
        status = 0
        for train_pano in data[train]:
            
            train_pano_image = train_pano.get_pano_image()
            for test_pano in data[test]:
                for test_label in test_pano.labels:
                    
                    template = test_label.get_label_crop_image()
                    result = match_template(train_pano_image, template)
                    x_pred, y_pred = np.unravel_index(np.argmax(result), result.shape)[::-1]

                    for train_label in train_pano.labels:
                        is_same_label = compare_labels(train_label, test_label)
                        distance = train_label.get_distance(x_pred, y_pred)
                        row_data = {
                            'target_pano_id' : train_pano.name,
                            'test_pano_id'   : test_pano.name,
                            'target_label_id': train_label.id,
                            'test_label_id'  : test_label.id,
                            'target_x'       : train_label.x,
                            'target_y'       : train_label.y,
                            'test_x_pred'    : x_pred,
                            'test_y_pred'    : y_pred,
                            'target_type'    : train_label.type,
                            'test_type'      : test_label.type,
                            'distance'       : distance,
                            'is_same_type'  : is_same_label
                        }
                        writer.writerow(row_data)
            status += 1
            print("Status: {} of {} test panos completed.".format(status, len(train)))
        print("Fold {} of 2 complete".format(k))
                    

                        

Status: 1 of 50 test panos completed.
Status: 2 of 50 test panos completed.
Status: 3 of 50 test panos completed.
Status: 4 of 50 test panos completed.
Status: 5 of 50 test panos completed.
Status: 6 of 50 test panos completed.
Status: 7 of 50 test panos completed.
Status: 8 of 50 test panos completed.
Status: 9 of 50 test panos completed.
Status: 10 of 50 test panos completed.
Status: 11 of 50 test panos completed.
Status: 12 of 50 test panos completed.
Status: 13 of 50 test panos completed.
Status: 14 of 50 test panos completed.
Status: 15 of 50 test panos completed.
Status: 16 of 50 test panos completed.
Status: 17 of 50 test panos completed.
Status: 18 of 50 test panos completed.
Status: 19 of 50 test panos completed.
Status: 20 of 50 test panos completed.
Status: 21 of 50 test panos completed.
Status: 22 of 50 test panos completed.
Status: 23 of 50 test panos completed.
Status: 24 of 50 test panos completed.
Status: 25 of 50 test panos completed.
Status: 26 of 50 test panos comple